<a href="https://colab.research.google.com/github/kategavrishina/info-search/blob/master/HW3/sem3_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лекция 3  NER

### __Задача 1__:

Реализуйте 2 функции препроцессинга:

- Удалить именованные сущности с помощью natasha (https://github.com/natasha/yargy)
- Удалить именованные сущности с помощью deepmipt (https://github.com/deepmipt/ner)

In [ ]:
! pip install natasha

In [ ]:
! pip install deeppavlov

In [12]:
text1 = """Школа анализа данных за 25 млрд долларов возникла в 2007 году на улице Крылова, д. 6 с целью восполнить недостаток квалифицированных кадров на рынке труда, с которым столкнулся Яндекс в то время. 24.02.13 Научным руководителем ШАД стал профессор Ратгерского университета Илья Мучник, директором — Елена Бунина."""

In [2]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()

morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [3]:
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [4]:
extractors = [
              dates_extractor,
              money_extractor,
              addr_extractor
]

def preprocess_with_natasha(text: str) -> str:

    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    indices = []
    for span in doc.spans:
        if span.type == "PER":
            indices.append((span.start, span.stop))
    for pair in sorted(indices)[::-1]:
        text = text[:pair[0]] + text[pair[1]:]

    for extractor in extractors:
        idx = []
        for match in list(extractor(text)):
            idx.append((match.start, match.stop))
        for pair in sorted(idx)[::-1]:
            text = text[:pair[0]] + text[pair[1]:]
    
    return text

In [5]:
from deeppavlov import configs, build_model

In [ ]:
! python -m deeppavlov install ner_rus_bert

In [ ]:
ner_model = build_model(configs.ner.ner_rus_bert, download=True)

In [11]:
! pip install razdel

In [6]:
from razdel import sentenize

def preprocess_with_deeppavlov(text: str) -> str:
    clean_text = []
    sents = [j.text for j in list(sentenize(text))]
    result = ner_model(sents)
    for i in range(len(result[0])):
        for word, tag in zip(result[0][i], result[1][i]):
            if tag == 'O':
                clean_text.append(word)
    return ' '.join(clean_text)

In [7]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

In [10]:
def preprocess(f):
    sym = "0123456789.,?!…:;()[]-—_|/\"'«»*{}<>@#$%^&№"
    s = []
    words = f.strip().lower().split()
    for word in words:
        word = word.strip(sym)
        if word != '':
            word = morph.parse(word)[0].normal_form
            if word not in russian_stopwords:
                s.append(word)
    return s

In [13]:
preprocess_with_natasha(text1)

'Школа анализа данных за  возникла в  на ,  целью восполнить недостаток квалифицированных кадров на рынке труда, с которым столкнулся Яндекс в то время.  Научным руководителем ШАД стал профессор Ратгерского университета , директором — .'

In [ ]:
preprocess_with_deeppavlov(text1)

### __Задача 2__:    
На предыдущем занятии вы реализовывали функции поиска ближайших ответов на запросы через TF-IDF и BM25. 
Сравните качество нахождения верного ответа для обоих методов в трех случаях:
- с функцией ```preprocess_with_natasha```
- с функцией ```preprocess_with_deepmipt```
- без препроцессинга

Для измерения качества используйте метрику accuracy. Считаем, что ответ верный, если он входит в топ-1.

In [15]:
import pandas as pd

In [24]:
ans = pd.read_excel('answers_base.xlsx')

In [25]:
ans.head()

,Номер связки,Текст вопросов,Текст ответа,Тематика
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ


In [26]:
answers = pd.DataFrame(ans[['Номер связки', 'Текст ответа']])

In [27]:
ans['Вопросы'] = ans['Текст вопросов'].apply(lambda text: text.split('\n'))

In [28]:
ans = ans.explode('Вопросы')

In [29]:
ans.reset_index(drop=True, inplace=True)
ans.drop(['Текст вопросов', 'Текст ответа', 'Тематика'], axis=1, inplace=True)

In [30]:
ans.head()

,Номер связки,Вопросы
0,57,У ребенка в школе продлили каникулы. Могу ли я...
1,57,Больничный лист?
2,57,"Есть ли компенсация, в случае если есть разниц..."
3,57,как оплачивается больничный при коронавирусе?
4,57,"Я контактный, дадут ли больничный?"


In [31]:
q = pd.read_excel('queries_base.xlsx', usecols='A, B', names=['Вопросы', 'Номер связки'])

In [32]:
q = q[['Номер связки', 'Вопросы']]

In [33]:
data = pd.concat([ans, q], ignore_index=True)

In [34]:
data['Preprocessed'] = data['Вопросы'].apply(lambda text: ' '.join(preprocess(str(text))))

In [26]:
from tqdm.auto import tqdm

In [27]:
tqdm.pandas()

In [37]:
data['Natasha'] = data['Вопросы'].progress_apply(lambda text: ' '.join(preprocess(preprocess_with_natasha(str(text)))))

In [38]:
data.head()

,Номер связки,Вопросы,Preprocessed,Natasha
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,ребёнок школа продлить каникулы мочь взять бол...,ребёнок школа продлить каникулы мочь взять бол...
1,57.0,Больничный лист?,больничный лист,больничный лист
2,57.0,"Есть ли компенсация, в случае если есть разниц...",компенсация случай разница оплата больничный з...,компенсация случай разница оплата больничный з...
3,57.0,как оплачивается больничный при коронавирусе?,оплачиваться больничный коронавирус,оплачиваться больничный коронавирус
4,57.0,"Я контактный, дадут ли больничный?",контактный дать больничный,контактный дать больничный


In [39]:
data['DeepPavlov'] = data['Вопросы'].progress_apply(lambda text: ' '.join(preprocess(preprocess_with_deeppavlov(str(text)))))

In [40]:
data.head()

,Номер связки,Вопросы,Preprocessed,Natasha,DeepPavlov
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,ребёнок школа продлить каникулы мочь взять бол...,ребёнок школа продлить каникулы мочь взять бол...,ребёнок школа продлить каникулы мочь взять бол...
1,57.0,Больничный лист?,больничный лист,больничный лист,больничный лист
2,57.0,"Есть ли компенсация, в случае если есть разниц...",компенсация случай разница оплата больничный з...,компенсация случай разница оплата больничный з...,компенсация случай разница оплата больничный з...
3,57.0,как оплачивается больничный при коронавирусе?,оплачиваться больничный коронавирус,оплачиваться больничный коронавирус,оплачиваться больничный коронавирус
4,57.0,"Я контактный, дадут ли больничный?",контактный дать больничный,контактный дать больничный,контактный дать больничный


In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [55]:
def make_tfidf(corpus):
    vectorizer = TfidfVectorizer(stop_words=russian_stopwords)
    X = vectorizer.fit_transform(corpus)
    tfidf = pd.DataFrame(X.A, columns=vectorizer.get_feature_names())
    return tfidf, vectorizer


def get_similar(query, vectorizer, tfidf_matrix, corpus):
    query_tfidf = vectorizer.transform([query]).toarray()
    result = tfidf_matrix.to_numpy().dot(query_tfidf.transpose())
    for i in np.argsort(result, axis=0)[::-1].transpose()[0]:
        return corpus.iloc[i]['Номер связки']

In [56]:
data_train, data_test = data[:2200], data[2200:]

In [ ]:
tfidf_p, vec_p = make_tfidf(data_train.Preprocessed)
data_test['TfIdf_Pred'] = data_test.apply(lambda x: get_similar(x.Preprocessed, vec_p, tfidf_p, data_train), axis=1)

In [ ]:
tfidf_n, vec_n = make_tfidf(data_train.Natasha)
data_test['TfIdf_Natasha_Pred'] = data_test.apply(lambda x: get_similar(x.Natasha, vec_n, tfidf_n, data_train), axis=1)

In [ ]:
tfidf_d, vec_d = make_tfidf(data_train.DeepPavlov)
data_test['TfIdf_DP_Pred'] = data_test.apply(lambda x: get_similar(x.DeepPavlov, vec_d, tfidf_d, data_train), axis=1)

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
from math import log

In [66]:
def bm25(item, avgdl, ld, k=2.0, b=0.75) -> float:
    score = (item * (k + 1)) / (item + k * (1 - b + ((b * ld) / avgdl)))
    return score


def get_bm(N, tf, avgdl, ld):
    bm = np.zeros((N, tf.shape[1]))
    for i in range(N):
        bm[i] = [bm25(word[i], avgdl, ld[i]) for word in tf.T]
    return bm


def get_similar_bm(query, vectorizer, corpus, bm, idf):
    query_vec = vectorizer.transform([query]).toarray() * idf
    result = bm.dot(query_vec.transpose())
    
    for i in np.argsort(result, axis=0)[::-1].transpose()[0]:
        return corpus.iloc[i]['Номер связки']

In [67]:
k = 2.0
b = 0.75

In [68]:
data_train.Preprocessed.fillna('', inplace=True)

corpus = data_train.Preprocessed

count_vectorizer = CountVectorizer()
tf = count_vectorizer.fit_transform(corpus).toarray()

N = len(tf)
lens = 0
ld = []
for text in corpus:
    l = len(text.split())
    lens += l
    ld.append(l)
avgdl = lens / N
ld = np.array(ld)

n_qi = [len(word.nonzero()[0]) for word in tf.T]
idf = [log((N - n_qi[i] + 0.5)/(n_qi[i] + 0.5)) for i in range(len(n_qi))]

In [69]:
bm = get_bm(N, tf, avgdl, ld)

In [70]:
bm.shape

(2200, 6029)

In [71]:
data_test['BM25_Pred'] = data_test.apply(lambda x: get_similar_bm(x.Preprocessed, count_vectorizer, data_train, bm, idf), axis=1)

In [72]:
data_train.Natasha.fillna('', inplace=True)

corpus_n = data_train.Natasha

count_vectorizer_n = CountVectorizer()
tf_n = count_vectorizer_n.fit_transform(corpus_n).toarray()

N_n = len(tf_n)
lens_n = 0
ld_n = []
for text in corpus_n:
    l = len(text.split())
    lens_n += l
    ld_n.append(l)
avgdl_n = lens_n / N_n
ld_n = np.array(ld_n)

n_qi_n = [len(word.nonzero()[0]) for word in tf_n.T]
idf_n = [log((N_n - n_qi_n[i] + 0.5)/(n_qi_n[i] + 0.5)) for i in range(len(n_qi_n))]

In [73]:
bm_n = get_bm(N_n, tf_n, avgdl_n, ld_n)

In [74]:
bm_n.shape

(2200, 5237)

In [75]:
data_test['BM25_Natasha_Pred'] = data_test.apply(lambda x: get_similar_bm(x.Natasha, count_vectorizer_n, data_train, bm_n, idf_n), axis=1)

In [76]:
data_train.DeepPavlov.fillna('', inplace=True)

corpus_d = data_train.DeepPavlov

count_vectorizer_d = CountVectorizer()
tf_d = count_vectorizer_d.fit_transform(corpus_d).toarray()

N_d = len(tf_d)
lens_d = 0
ld_d = []
for text in corpus_d:
    l = len(text.split())
    lens_d += l
    ld_d.append(l)
avgdl_d = lens_d / N_d
ld_d = np.array(ld_d)

n_qi_d = [len(word.nonzero()[0]) for word in tf_d.T]
idf_d = [log((N_d - n_qi_d[i] + 0.5)/(n_qi_d[i] + 0.5)) for i in range(len(n_qi_d))]

In [77]:
bm_d = get_bm(N_d, tf_d, avgdl_d, ld_d)

In [78]:
bm_d.shape

(2200, 4517)

In [79]:
data_test['BM25_DP_Pred'] = data_test.apply(lambda x: get_similar_bm(x.DeepPavlov, count_vectorizer_d, data_train, bm_d, idf_d), axis=1)

Accuracy для TF-IDF без удаления NER:

In [81]:
len(data_test[data_test['Номер связки'] == data_test.TfIdf_Pred]) / len(data_test)

0.5136363636363637

Accuracy для TF-IDF после удаления NER с помощью Natasha:

In [82]:
len(data_test[data_test['Номер связки'] == data_test.TfIdf_Natasha_Pred]) / len(data_test)

0.5

Accuracy для TF-IDF после удаления NER с помощью DeepPavlov:

In [83]:
len(data_test[data_test['Номер связки'] == data_test.TfIdf_DP_Pred]) / len(data_test)

0.4909090909090909

Accuracy для BM25 без удаления NER

In [84]:
len(data_test[data_test['Номер связки'] == data_test.BM25_Pred]) / len(data_test)

0.5511363636363636

Accuracy для BM25 после удаления NER с помощью Natasha

In [85]:
len(data_test[data_test['Номер связки'] == data_test.BM25_Natasha_Pred]) / len(data_test)

0.5511363636363636

Accuracy для BM25 после удаления NER с помощью DeepPavlov

In [86]:
len(data_test[data_test['Номер связки'] == data_test.BM25_DP_Pred]) / len(data_test)

0.5363636363636364

### __Задача 3__:    
Улучшить правила в natasha. Написать правила, которые ловят даты в следующих примерах и пересчитать статистику из Задачи 2:
- Уехал 8-9 ноября в Сочи
- Уезжаю 5 числа                           
- 20го сентября заболел

Пример можно посмотреть тут: https://github.com/natasha/yargy

In [87]:
! pip install yargy

In [88]:
from yargy import Parser, rule, and_, or_
from yargy.interpretation import fact
from yargy.predicates import (
    lte,
    gte,
    caseless,
    dictionary
)

Date = fact(
    'Date',
    ['year', 'month', 'day']
)

MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'май',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}

MONTH_NAME = dictionary(MONTHS)
DAY = and_(
    gte(1),
    lte(31)
)
DATE = or_(
    rule(
        DAY.interpretation(
            Date.day
            ),
        'го',
        MONTH_NAME.interpretation(
            Date.month
        )
    ),
    rule(
        DAY.interpretation(
            Date.day
            ),
         caseless('числа')
    ),
    rule(
        DAY.interpretation(
            Date.day
            ),
        '-',
        DAY.interpretation(
            Date.day
            ),
        MONTH_NAME.interpretation(
            Date.month
        )
    )
).named('DATE')
parser = Parser(DATE)

In [90]:
import yargy
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()

morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [91]:
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [109]:
extractors = [
              dates_extractor,
              money_extractor,
              addr_extractor,
]

def preprocess_with_natasha(text: str) -> str:

    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    id = []
    for match in parser.findall(text):
        id.append((match.span.start, match.span.stop))
    for pair in sorted(id)[::-1]:
        text = text[:pair[0]] + text[pair[1]:]

    indices = []
    for span in doc.spans:
        if span.type == "PER":
            indices.append((span.start, span.stop))
    for pair in sorted(indices)[::-1]:
        text = text[:pair[0]] + text[pair[1]:]

    for extractor in extractors:
        idx = []
        for match in list(extractor(text)):
            idx.append((match.start, match.stop))
        for pair in sorted(idx)[::-1]:
            text = text[:pair[0]] + text[pair[1]:]
    
    return text

In [112]:
data_train['Natasha_New'] = data_train['Вопросы'].progress_apply(lambda text: ' '.join(preprocess(preprocess_with_natasha(str(text)))))

In [113]:
data_test['Natasha_New'] = data_test['Вопросы'].progress_apply(lambda text: ' '.join(preprocess(preprocess_with_natasha(str(text)))))

In [114]:
tfidf_nn, vec_nn = make_tfidf(data_train.Natasha_New)
data_test['TfIdf_Natasha_New_Pred'] = data_test.apply(lambda x: get_similar(x.Natasha_New, vec_nn, tfidf_nn, data_train), axis=1)

In [115]:
data_train.Natasha_New.fillna('', inplace=True)

corpus_nn = data_train.Natasha_New

count_vectorizer_nn = CountVectorizer()
tf_nn = count_vectorizer_nn.fit_transform(corpus_nn).toarray()

N_nn = len(tf_nn)
lens_nn = 0
ld_nn = []
for text in corpus_nn:
    l = len(text.split())
    lens_nn += l
    ld_nn.append(l)
avgdl_nn = lens_nn / N_nn
ld_nn = np.array(ld_nn)

n_qi_nn = [len(word.nonzero()[0]) for word in tf_nn.T]
idf_nn = [log((N_nn - n_qi_nn[i] + 0.5)/(n_qi_nn[i] + 0.5)) for i in range(len(n_qi_nn))]

In [116]:
bm_nn = get_bm(N_nn, tf_nn, avgdl_nn, ld_nn)

In [117]:
bm_nn.shape

(2200, 5255)

In [118]:
data_test['BM25_Natasha_New_Pred'] = data_test.apply(lambda x: get_similar_bm(x.Natasha_New, count_vectorizer_nn, data_train, bm_nn, idf_nn), axis=1)

In [119]:
data_test.sample(10)

,Номер связки,Вопросы,Preprocessed,Natasha,DeepPavlov,TfIdf_Pred,TfIdf_Natasha_Pred,TfIdf_DP_Pred,BM25_Pred,BM25_Natasha_Pred,BM25_DP_Pred,Natasha_New,TfIdf_Natasha_New_Pred,BM25_Natasha_New_Pred
2992,1.0,Добрый день. Сегодня на моего ребенка в школу ...,добрый день сегодня ребёнок школа поступить пи...,добрый день сегодня ребёнок школа поступить пи...,добрый день сегодня ребёнок школа поступить пи...,1,257.0,257.0,1.0,1.0,1.0,добрый день сегодня ребёнок школа поступить пи...,257.0,1.0
2558,1.0,"Здравствуйте, у меня положительный тест на cov...",здравствовать положительный тест covid нужно в...,здравствовать положительный тест covid нужно в...,здравствовать положительный тест covid нужно в...,57,57.0,57.0,12.0,12.0,12.0,здравствовать положительный тест covid нужно в...,57.0,12.0
2869,308.0,\nТогда ещё вопрос при вылете из РФ в я Турцию...,ещё вопрос вылет рф турция нужно сдавать анали...,ещё вопрос вылет турция нужно сдавать анализ к...,ещё вопрос вылет нужно сдавать анализ ковид от...,308,308.0,308.0,308.0,308.0,308.0,ещё вопрос вылет турция нужно сдавать анализ к...,308.0,308.0
2767,308.0,ДОбрый день!\n\nя прилетел в воскресенье рано ...,добрый день прилететь воскресение рано утром ш...,добрый день прилететь воскресение рано утром ш...,добрый день прилететь воскресение рано утром п...,308,308.0,308.0,308.0,308.0,308.0,добрый день прилететь воскресение рано утром ш...,308.0,308.0
2864,308.0,Добрый день. Ребенок 3 года - нужен ли тест на...,добрый день ребёнок год нужный тест ковид возв...,добрый день ребёнок год нужный тест ковид возв...,добрый день ребёнок год нужный тест ковид возв...,12,12.0,12.0,12.0,308.0,12.0,добрый день ребёнок год нужный тест ковид возв...,12.0,308.0
2922,308.0,Добрый день!\n\nРазъясните пожалуйста такой мо...,добрый день разъяснить пожалуйста момент семья...,добрый день разъяснить пожалуйста момент семья...,добрый день разъяснить пожалуйста момент семья...,308,308.0,308.0,308.0,308.0,308.0,добрый день разъяснить пожалуйста момент семья...,308.0,308.0
2511,308.0,Здравствуйте!\nЯ уезжаю в отпуск в Абхазию в с...,здравствовать уезжать отпуск абхазия сентябрь ...,здравствовать уезжать отпуск абхазия возвращен...,здравствовать уезжать отпуск сентябрь год возв...,308,308.0,308.0,286.0,286.0,37.0,здравствовать уезжать отпуск абхазия возвращен...,308.0,286.0
2676,308.0,Добрый день!\nВчера 30.08.2020 я прилетел из Т...,добрый день вчера прилететь турция аэропорт пу...,добрый день вчера прилететь турция аэропорт пу...,добрый день вчера прилететь аэропорт финансовы...,12,12.0,12.0,12.0,12.0,1.0,добрый день вчера прилететь турция аэропорт пу...,12.0,12.0
2758,308.0,"Это касается\nГраждан Российской Федерации, пр...",это касаться гражданин российский федерация пр...,это касаться гражданин российский федерация пр...,это касаться гражданин прибывать территория во...,308,308.0,308.0,308.0,308.0,308.0,это касаться гражданин российский федерация пр...,308.0,308.0
2848,308.0,Добрый день! Вчера я с ребенком вернулась из-з...,добрый день вчера ребёнок вернуться из-за гран...,добрый день вчера ребёнок вернуться из-за гран...,добрый день вчера ребёнок вернуться граница де...,308,308.0,308.0,308.0,308.0,308.0,добрый день вчера ребёнок вернуться из-за гран...,308.0,308.0


Accuracy для Tf-Idf после удаления NER с помощью Natasha с новыми правилами

In [120]:
len(data_test[data_test['Номер связки'] == data_test.TfIdf_Natasha_New_Pred]) / len(data_test)

0.5

Accuracy для BM25 после удаления NER с помощью Natasha с новыми правилами

In [121]:
len(data_test[data_test['Номер связки'] == data_test.BM25_Natasha_New_Pred]) / len(data_test)

0.5511363636363636

Показатели лучше не стали, но некоторые ответы все-таки поменялись

In [125]:
len(data_test[data_test.BM25_Natasha_Pred != data_test.BM25_Natasha_New_Pred])

4

In [126]:
len(data_test[data_test.TfIdf_Natasha_Pred != data_test.TfIdf_Natasha_New_Pred])

3